In [3]:
import pandas as pd


In [8]:
leagues = [
    "eng-premier_league",  # Premier League#
    "esp-la_liga",  # La Liga
    "ita-serie_a",  # Serie A
    "ger-bundesliga",  # Bundesliga
    "fra-ligue_1",  # Ligue 1
]

advanced_stats = ["schedule","shooting","passing", "goal_shot_creation"
"possession", "defense", "passing_types"]


In [16]:
all_years_schedule = pd.DataFrame()
for league in leagues: 
    # path is /Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/ger-bundesliga_schedule_2015-2016_2016-2017_2017-2018_2018-2019_2019-2020_2020-2021_2021-2022_2022-2023_2023-2024_2024-2025.csv
    # /Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/fra-ligue_1_schedule_2015-2016_2016-2017_2017-2018_2018-2019_2019-2020_2020-2021_2021-2022_2022-2023_2023-2024_2024-2025.csv
    # /Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/ita-serie_a_schedule_2015-2016_2016-2017_2017-2018_2018-2019_2019-2020_2020-2021_2021-2022_2022-2023_2023-2024_2024-2025.csv
    # /Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/esp-la_liga_schedule_2015-2016_2016-2017_2017-2018_2018-2019_2019-2020_2020-2021_2021-2022_2022-2023_2023-2024_2024-2025.csv
    
    # /Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/eng-premier_league_schedule_2015-2016_2016-2017_2017-2018_2018-2019_2019-2020_2020-2021_2021-2022_2022-2023_2023-2024_2024-2025.csv
    # Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/eng-premier_league_schedule_schedule_2015-2016_2016-2017_2017-2018_2018-2019_2019-2020_2020-2021_2021-2022_2022-2023_2023-2024_2024-2025.csv'
    schedule_data = pd.read_csv(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/{league.lower()}_schedule_2015-2016_2016-2017_2017-2018_2018-2019_2019-2020_2020-2021_2021-2022_2022-2023_2023-2024_2024-2025.csv")
    schedule_data["home_team_goals"] =  schedule_data["score"].str.split("–", expand=True)[0]
    schedule_data["away_team_goals"] =  schedule_data["score"].str.split("–", expand=True)[1]
    schedule_data["league"] = league

    # define the mapping dictionary
    schedule = {"week": "week","date": "date", "home_team": "home_team", "away_team": "away_team", 
            "home_team_goals": "home_team_goals", "away_team_goals": "away_team_goals", "match_report": "match_report",
            "home_xg": "home_xg", "away_xg": "away_xg"}
    # restrict and rename columns
    schedule_data = schedule_data[list(schedule.keys())].rename(columns=schedule)
    schedule_data["league"] = league
    # safe as pickle 
    # all years

    all_years_schedule = pd.concat([all_years_schedule, schedule_data], ignore_index=True) if 'all_years_schedule' in locals() else schedule_data

all_years_schedule.to_pickle(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/schedule_all_years.pkl")




    




In [29]:
years = ["2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020",
         "2020-2021", "2021-2022", "2022-2023", "2023-2024", "2024-2025", "2025-2026"]
defense_data_cols = {"Int": "int", "Blocks": "blocks", "Clr": "clr", "Err": "err", "season": "season"}
all_years_defense = pd.DataFrame()
all_leagues_defense = pd.DataFrame()

# delete the first unname
# d row 
for league in leagues:
    for year in years:
        # eng-premier-league_defense_2015-2016.csv should be the file name
        defense_data = pd.read_csv(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/{league.lower()}_defense_{year}.csv")
        defense_data["season"] = year
        defense_data
        defense_data = defense_data.iloc[1:]                            
        defense_data_home = defense_data[defense_data["venue"] == "Home"]
        defense_data_away = defense_data[defense_data["venue"] == "Away"]
        defense_data_merged = pd.merge(defense_data_home[["match_report"] + list(defense_data_cols.keys())],
                                    defense_data_away[["match_report"] + list(defense_data_cols.keys())],
                                    on="match_report",
                                suffixes=("_home", "_away"))

        
        # delete season away and rename season home to season
        defense_data_merged = defense_data_merged.drop(columns=["season_away"])
        defense_data_merged = defense_data_merged.rename(columns={"season_home": "season"})
        # all years merged 
        all_years_defense = pd.concat([all_years_defense, defense_data_merged], ignore_index=True) if 'all_years_defense' in locals() else defense_data_merged
    all_leagues_defense = pd.concat([all_years_defense, defense_data_merged], ignore_index=True) if 'all_years_defense' in locals() else defense_data_merged
# safe as pickle
all_leagues_defense.to_pickle(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/defense_data_merged.pkl")


In [ ]:
passing = {"Total": "pas_tot", "Total.1" : "pas_att", "Total.4": "pgrssv_dist", "Total.3": "tot_dist", "Ast": "ast",
           "xAG": "exp_assists_g", "xA": "exp_assists"}
passing_all_years = pd.DataFrame()  
passing_all_leagues = pd.DataFrame()
for league in leagues:
    for year in years:
        passing_data = pd.read_csv(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/{league.lower()}_passing_{year}.csv")
        #delete the first unnamed row
        passing_data = passing_data.iloc[1:]
        # ensure that all columns are in there, if not add them with NaN values
        for col in passing.keys():
            if col not in passing_data.columns:
                passing_data[col] = pd.NA
        passing_data_home = passing_data[passing_data["venue"] == "Home"]
        passing_data_away = passing_data[passing_data["venue"] == "Away"]
        passing_data_merged = pd.merge(passing_data_home[["match_report"] + list
                                        (passing.keys())],
                                    passing_data_away[["match_report"] + list(passing.keys())],
                                    on="match_report",
                                    suffixes=("_home", "_away"))
        # rename 
        passing_data_merged = passing_data_merged.rename(columns={k + "_home": v + "_home" for k, v in passing.items()})
        passing_data_merged = passing_data_merged.rename(columns={k + "_away": v + "_away" for k, v in passing.items()})
        passing_data_merged["season"] = year
        # merge all years
        passing_all_years = pd.concat([passing_all_years, passing_data_merged], ignore_index=True) if 'passing_all_years' in locals() else passing_data_merged
    passing_all_leagues = pd.concat([passing_all_leagues, passing_all_years], ignore_index=True) if 'passing_all_leagues' in locals() else passing_all_years
# save as pickle
passing_all_leagues.to_pickle(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/passing_data_merged.pkl")

/var/folders/k1/n035yq891w95hzqpmlll09k80000gn/T/ipykernel_91519/4008968258.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  passing_all_years = pd.concat([passing_all_years, passing_data_merged], ignore_index=True) if 'passing_all_years' in locals() else passing_data_merged
/var/folders/k1/n035yq891w95hzqpmlll09k80000gn/T/ipykernel_91519/4008968258.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  passing_all_years = pd.concat([passing_all_years, passing_data_merged], ignore_index=True) 

In [36]:
passing_type ={"Pass Types.5": "crosses"}
passing_types_all_leagues = pd.DataFrame()
passing_types_all_years = pd.DataFrame()
for league in leagues:  
    for year in years:
        passing_data = pd.read_csv(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/{league.lower()}_passing_types_{year}.csv")
        # delete the first unnamed row
        passing_data = passing_data.iloc[1:]
        # ensure that all columns are in there, if not add them with NaN values
        for col in passing_type.keys():
            if col not in passing_data.columns:
                passing_data[col] = pd.NA
        passing_data_home = passing_data[passing_data["venue"] == "Home"]
        passing_data_away = passing_data[passing_data["venue"] == "Away"]
        passing_data_merged = pd.merge(passing_data_home[["match_report"] + list
                                        (passing_type.keys())],
                                    passing_data_away[["match_report"] + list(passing_type.keys())],
                                    on="match_report",
                                    suffixes=("_home", "_away"))
        # rename 
        passing_data_merged = passing_data_merged.rename(columns={k + "_home": v + "_home" for k, v in passing_type.items()})
        passing_data_merged = passing_data_merged.rename(columns={k + "_away": v + "_away" for k, v in passing_type.items()})
        passing_data_merged["season"] = year
        # merge all years
        passing_types_all_years = pd.concat([passing_types_all_years, passing_data_merged], ignore_index=True) if 'passing_types_all_years' in locals() else passing_data_merged
    passing_types_all_leagues = pd.concat([passing_types_all_leagues, passing_types_all_years], ignore_index=True) if 'passing_types_all_leagues' in locals() else passing_types_all_years
# save as pickle
# delete duplicates 
passing_types_all_leagues = passing_types_all_leagues.drop_duplicates()
passing_types_all_leagues.to_pickle(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/passing_types_data_merged.pkl")

In [40]:
shots = {"Standard.2":"SoT", "Expected": "xg","Standard.3":"SoT_over_goals", 
         "Expected.1": "npxg", "Expected.2": "npxg_per_shot"}

shots_all_leagues = pd.DataFrame()
shots_all_years = pd.DataFrame()
for league in leagues:
    for year in years:
        shots_data = pd.read_csv(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/fbref/{league.lower()}_shooting_{year}.csv")
        # delete the first unnamed row
        shots_data = shots_data.iloc[1:]
        # ensure that all columns are in there, if not add them with NaN values
        for col in shots.keys():
            if col not in shots_data.columns:
                shots_data[col] = pd.NA
        shots_data_home = shots_data[shots_data["venue"] == "Home"]
        shots_data_away = shots_data[shots_data["venue"] == "Away"]
        shots_data_merged = pd.merge(shots_data_home[["match_report"] + list
                                        (shots.keys())],
                                    shots_data_away[["match_report"] + list(shots.keys())],
                                    on="match_report",
                                    suffixes=("_home", "_away"))
        # rename 
        shots_data_merged = shots_data_merged.rename(columns={k + "_home": v + "_home" for k, v in shots.items()})
        shots_data_merged = shots_data_merged.rename(columns={k + "_away": v + "_away" for k, v in shots.items()})
        shots_data_merged["season"] = year
        # merge all years
        shots_all_years = pd.concat([shots_all_years, shots_data_merged], ignore_index=True) if 'shots_all_years' in locals() else shots_data_merged
    shots_all_leagues = pd.concat([shots_all_leagues, shots_all_years], ignore_index=True) if 'shots_all_leagues' in locals() else shots_all_years
# save as pickle
# delete duplicates
shots_all_leagues = shots_all_leagues.drop_duplicates()
shots_all_leagues.to_pickle(f"/Users/luisenriquekaiser/Documents/soccer_betting_forecast/data/processed/shooting_data_merged.pkl")


In [41]:
shots_all_leagues

,match_report,SoT_home,xg_home,SoT_over_goals_home,npxg_home,npxg_per_shot_home,SoT_away,xg_away,SoT_over_goals_away,npxg_away,npxg_per_shot_away,season
0,/en/matches/a64b94e8/Crystal-Palace-Arsenal-Au...,4,<NA>,36.4,<NA>,<NA>,7,<NA>,35.0,<NA>,<NA>,2015-2016
1,/en/matches/833982d1/Newcastle-United-Arsenal-...,0,<NA>,0.0,<NA>,<NA>,9,<NA>,40.9,<NA>,<NA>,2015-2016
2,/en/matches/66a9fa0a/North-West-London-Derby-C...,6,<NA>,27.3,<NA>,<NA>,2,<NA>,22.2,<NA>,<NA>,2015-2016
3,/en/matches/69e04a8b/Leicester-City-Arsenal-Se...,7,<NA>,43.8,<NA>,<NA>,12,<NA>,44.4,<NA>,<NA>,2015-2016
4,/en/matches/483b03ee/Watford-Arsenal-October-1...,1,<NA>,12.5,<NA>,<NA>,6,<NA>,35.3,<NA>,<NA>,2015-2016
...,...,...,...,...,...,...,...,...,...,...,...,...
57334,/en/matches/a2ea8dfd/Auxerre-Toulouse-Septembe...,1,0.7,16.7,0.7,0.11,2,2.2,16.7,1.5,0.12,2025-2026
57335,/en/matches/60ecc64f/Lyon-Toulouse-October-5-2...,3,1.2,23.1,1.2,0.09,3,1.2,21.4,1.2,0.08,2025-2026
57336,/en/matches/fe7036bb/Monaco-Toulouse-October-2...,5,2.9,35.7,2.9,0.23,2,0.7,20.0,0.7,0.07,2025-2026
57337,/en/matches/3b9a668d/Lorient-Toulouse-November...,1,1.3,11.1,0.6,0.06,1,1.3,20.0,0.5,0.11,2025-2026
